In [2]:
import numpy as np
import pandas as pd
import glob

In [21]:
pd.set_option('display.max_columns', 50)

In [25]:
meta = pd.read_csv('../data/raw/toil_phenotype_260422.txt', sep='\t', encoding='windows-1251')
gdc = pd.read_excel('../data/custom/gdc_tcga_study_040522.xlsx')

In [ ]:
surv = pd.read_csv('../data/raw/toil_')

In [26]:
meta['_study'].unique()

array(['TCGA', 'GTEX', 'TARGET'], dtype=object)

In [38]:
meta_tcga = meta[meta['_study'] == 'TCGA']

In [69]:
meta_gtex = meta[meta['_study'] == 'GTEX']

In [39]:
meta_tcga

,sample,detailed_category,primary disease or tissue,_primary_site,_sample_type,_gender,_study
0,TCGA-V4-A9EE-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
1,TCGA-VD-AA8N-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
2,TCGA-V4-A9EI-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
3,TCGA-VD-AA8O-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
4,TCGA-WC-A888-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
...,...,...,...,...,...,...,...
10530,TCGA-AB-2812-03,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Peripheral Blood,Female,TCGA
10531,TCGA-AB-2914-03,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Peripheral Blood,Female,TCGA
10532,TCGA-AB-2896-03,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Peripheral Blood,Female,TCGA
10533,TCGA-AB-3011-03,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Peripheral Blood,Female,TCGA


In [40]:
meta_tcga = meta_tcga.dropna()

In [41]:
meta_tcga.iloc[:, 1:-1] = meta_tcga.iloc[:, 1:-1].apply(lambda x: x.str.lower())

C:\Users\Dynev\AppData\Local\Temp\ipykernel_12196\1337632555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_tcga.iloc[:, 1:-1] = meta_tcga.iloc[:, 1:-1].apply(lambda x: x.str.lower())


In [42]:
meta_tcga = meta_tcga[meta_tcga['_sample_type'].isin(['primary tumor', 'solid tissue normal'])]

In [43]:
meta_tcga = meta_tcga.drop(['_study'], axis=1)

In [44]:
meta_tcga['project'] = meta_tcga['detailed_category'].map(gdc.set_index('full_name')['abbreviation'])

In [46]:
meta_tcga[meta_tcga.project.isna()]

,sample,detailed_category,primary disease or tissue,_primary_site,_sample_type,_gender,project


In [53]:
(meta_tcga['detailed_category'] == meta_tcga['primary disease or tissue']).all()

True

In [54]:
meta_tcga = meta_tcga.drop(['primary disease or tissue'], axis=1)

In [56]:
meta_tcga = meta_tcga.rename({'detailed_category': 'cancer_type', '_primary_site': 'primary_site', '_sample_type': 'sample_type', '_gender': 'gender'}, axis=1)

In [57]:
meta_tcga = meta_tcga[['project', 'cancer_type', 'primary_site', 'sample', 'gender', 'sample_type']]

In [59]:
meta_tcga['primary_site'].unique()

array(['eye', 'endometrium', 'uterus', 'thyroid gland', 'thymus',
       'testis', 'stomach', 'skin', 'soft tissue,bone', 'rectum',
       'prostate', 'paraganglia', 'pancreas', 'ovary',
       'lining of body cavities', 'lung', 'liver', 'kidney',
       'head and neck region', 'brain', 'esophagus', 'lymphatic tissue',
       'colon', 'bile duct', 'cervix', 'breast', 'bladder',
       'adrenal gland'], dtype=object)

In [75]:
meta_tcga['sample_type'] = meta_tcga['sample_type'].str.replace('solid tissue normal', 'normal tissue') 

In [76]:
meta_tcga

,project,cancer_type,primary_site,sample,gender,sample_type
0,UVM,uveal melanoma,eye,TCGA-V4-A9EE-01,male,primary tumor
1,UVM,uveal melanoma,eye,TCGA-VD-AA8N-01,male,primary tumor
2,UVM,uveal melanoma,eye,TCGA-V4-A9EI-01,male,primary tumor
3,UVM,uveal melanoma,eye,TCGA-VD-AA8O-01,male,primary tumor
4,UVM,uveal melanoma,eye,TCGA-WC-A888-01,male,primary tumor
...,...,...,...,...,...,...
10356,ACC,adrenocortical cancer,adrenal gland,TCGA-OR-A5J3-01,female,primary tumor
10357,ACC,adrenocortical cancer,adrenal gland,TCGA-OR-A5KY-01,female,primary tumor
10358,ACC,adrenocortical cancer,adrenal gland,TCGA-OR-A5J9-01,female,primary tumor
10359,ACC,adrenocortical cancer,adrenal gland,TCGA-OR-A5JE-01,female,primary tumor


In [70]:
meta_gtex = meta_gtex.dropna()

In [71]:
meta_gtex

,sample,detailed_category,primary disease or tissue,_primary_site,_sample_type,_gender,_study
10535,K-562-SM-57WAW,Cells - Leukemia Cell Line (Cml),Cells - Leukemia Cell Line (Cml),Bone Marrow,Cell Line,Female,GTEX
10536,K-562-SM-48U2C,Cells - Leukemia Cell Line (Cml),Cells - Leukemia Cell Line (Cml),Bone Marrow,Cell Line,Female,GTEX
10537,K-562-SM-47JYY,Cells - Leukemia Cell Line (Cml),Cells - Leukemia Cell Line (Cml),Bone Marrow,Cell Line,Female,GTEX
10538,K-562-SM-32PMP,Cells - Leukemia Cell Line (Cml),Cells - Leukemia Cell Line (Cml),Bone Marrow,Cell Line,Female,GTEX
10539,K-562-SM-5CVL5,Cells - Leukemia Cell Line (Cml),Cells - Leukemia Cell Line (Cml),Bone Marrow,Cell Line,Female,GTEX
...,...,...,...,...,...,...,...
18392,GTEX-POMQ-2326-SM-2S1O8,Adipose - Subcutaneous,Adipose - Subcutaneous,Adipose Tissue,Normal Tissue,Female,GTEX
18393,GTEX-QVJO-0226-SM-2S1R2,Adipose - Subcutaneous,Adipose - Subcutaneous,Adipose Tissue,Normal Tissue,Female,GTEX
18394,GTEX-P4PP-0226-SM-2S1NN,Adipose - Subcutaneous,Adipose - Subcutaneous,Adipose Tissue,Normal Tissue,Female,GTEX
18395,GTEX-NL3G-0226-SM-4RGLU,Adipose - Subcutaneous,Adipose - Subcutaneous,Adipose Tissue,Normal Tissue,Female,GTEX


In [72]:
meta_gtex.iloc[:, 1:-1] = meta_gtex.iloc[:, 1:-1].apply(lambda x: x.str.lower())

In [73]:
meta_gtex = meta_gtex[meta_gtex['_sample_type'] == 'normal tissue']

In [78]:
(meta_gtex['detailed_category'] == meta_gtex['primary disease or tissue']).all()

True

In [79]:
meta_gtex = meta_gtex.drop(['primary disease or tissue'], axis=1)

In [81]:
meta_gtex = meta_gtex.drop(['_sample_type', '_study'], axis=1)

In [83]:
meta_gtex = meta_gtex.rename({'detailed_category': 'detailed_site', '_primary_site': 'primary_site', '_gender': 'gender'}, axis=1)

In [84]:
meta_gtex = meta_gtex[['primary_site', 'detailed_site', 'sample', 'gender']]

In [87]:
meta_gtex['primary_site'] = meta_gtex['primary_site'].str.replace('cervix uteri', 'cervix')

In [95]:
meta_tcga_groups = pd.DataFrame(index=meta_tcga['project'].sort_values().unique())

In [96]:
meta_tcga_groups['cancer'] = meta_tcga_groups.index.map(meta_tcga[meta_tcga['sample_type'] == 'primary tumor'].groupby('project').size())

In [98]:
meta_tcga_groups['norm'] = meta_tcga_groups.index.map(meta_tcga[meta_tcga['sample_type'] == 'normal tissue'].groupby('project').size())

In [101]:
meta_tcga_groups['norm'] = meta_tcga_groups['norm'].fillna(0).astype(int)

In [112]:
grr = meta_tcga.groupby(['project', '_sample_stem']).size()

In [115]:
meta_tcga_groups['paired'] = grr[grr > 1].groupby('project').size()

In [105]:
meta_tcga['_sample_stem'] = meta_tcga['sample'].str[:-3]

In [104]:
meta_tcga[meta_tcga['sample_type'] == 'normal tissue']

,project,cancer_type,primary_site,sample,gender,sample_type
79,UCEC,uterine corpus endometrioid carcinoma,endometrium,TCGA-AJ-A3NH-11,female,normal tissue
80,UCEC,uterine corpus endometrioid carcinoma,endometrium,TCGA-BG-A3PP-11,female,normal tissue
81,UCEC,uterine corpus endometrioid carcinoma,endometrium,TCGA-AX-A2HD-11,female,normal tissue
82,UCEC,uterine corpus endometrioid carcinoma,endometrium,TCGA-DI-A2QY-11,female,normal tissue
83,UCEC,uterine corpus endometrioid carcinoma,endometrium,TCGA-AJ-A3NC-11,female,normal tissue
...,...,...,...,...,...,...
9872,BLCA,bladder urothelial carcinoma,bladder,TCGA-K4-A3WV-11,female,normal tissue
9873,BLCA,bladder urothelial carcinoma,bladder,TCGA-BT-A2LB-11,female,normal tissue
9874,BLCA,bladder urothelial carcinoma,bladder,TCGA-BT-A20U-11,female,normal tissue
9875,BLCA,bladder urothelial carcinoma,bladder,TCGA-K4-A5RI-11,female,normal tissue


In [117]:
meta_tcga_groups['paired'] = meta_tcga_groups['paired'].fillna(0).astype(int)

In [120]:
meta_tcga_groups['cancer_unpaired'] = meta_tcga_groups['cancer'] - meta_tcga_groups['paired']

In [122]:
meta_tcga_groups = meta_tcga_groups.rename({'norm': 'normal'}, axis=1)

In [124]:
meta_tcga_groups['normal_unpaired'] = meta_tcga_groups['normal'] - meta_tcga_groups['paired']

In [125]:
meta_tcga_groups['cancer_type'] = meta_tcga_groups.index.map(meta_tcga.drop_duplicates('project').set_index('project')['cancer_type'])

In [126]:
meta_tcga_groups['primary_site'] = meta_tcga_groups.index.map(meta_tcga.drop_duplicates('project').set_index('project')['primary_site'])

In [127]:
meta_tcga_groups = meta_tcga_groups.reset_index()

In [129]:
meta_tcga_groups = meta_tcga_groups.rename({'index': 'project'}, axis=1)

In [131]:
meta_tcga_groups = meta_tcga_groups[['project', 'cancer_type', 'primary_site', 'cancer', 'normal', 'paired', 'cancer_unpaired', 'normal_unpaired']]

In [133]:
meta_tcga_groups.to_csv('../data/processed/toil_tcga_group_size_140722.csv', index=False)

In [135]:
meta_tcga_groups[meta_tcga_groups['normal'] < 19]

,project,cancer_type,primary_site,cancer,normal,paired,cancer_unpaired,normal_unpaired
0,ACC,adrenocortical cancer,adrenal gland,77,0,0,77,0
3,CESC,cervical & endocervical cancer,cervix,304,3,3,301,0
4,CHOL,cholangiocarcinoma,bile duct,36,9,9,27,0
6,DLBC,diffuse large b-cell lymphoma,lymphatic tissue,47,0,0,47,0
7,ESCA,esophageal carcinoma,esophagus,181,13,13,168,0
8,GBM,glioblastoma multiforme,brain,152,0,0,152,0
13,LGG,brain lower grade glioma,brain,508,0,0,508,0
17,MESO,mesothelioma,lining of body cavities,87,0,0,87,0
18,OV,ovarian serous cystadenocarcinoma,ovary,419,0,0,419,0
19,PAAD,pancreatic adenocarcinoma,pancreas,178,4,4,174,0


In [138]:
meta_gtex.groupby('detailed_site').size()

detailed_site
adipose - subcutaneous                       318
adipose - visceral (omentum)                 197
adrenal gland                                128
artery - aorta                               207
artery - coronary                            118
artery - tibial                              281
bladder                                        9
brain - amygdala                              69
brain - anterior cingulate cortex (ba24)      83
brain - caudate (basal ganglia)              109
brain - cerebellar hemisphere                 97
brain - cerebellum                           119
brain - cortex                               105
brain - frontal cortex (ba9)                 102
brain - hippocampus                           84
brain - hypothalamus                          82
brain - nucleus accumbens (basal ganglia)    104
brain - putamen (basal ganglia)               81
brain - spinal cord (cervical c-1)            60
brain - substantia nigra                      57
breast

In [141]:
meta_tcga.to_csv('../data/processed/toil_tcga_sample_info_071422.csv', index=False)

In [142]:
meta_gtex.to_csv('../data/processed/toil_gtex_sample_info_071422.csv', index=False)

In [143]:
dep = pd.read_csv('../data/raw/depmap_22q1_cellline_info_130422.csv')

In [147]:
dep = dep.rename({'DepMap_ID': 'depmap_id', 'CCLE_Name': 'ccle_name', 'Subtype': 'disease_subtype'}, axis=1)

In [150]:
dep = dep[['depmap_id', 'cell_line_name', 'stripped_cell_line_name', 'ccle_name', 'sex', 'sample_collection_site', 'primary_or_metastasis', 'primary_disease',
       'disease_subtype', 'lineage', 'lineage_subtype', 'lineage_sub_subtype',
       'lineage_molecular_subtype', 'culture_type']]

In [159]:
dep['disease_subtype'].str.contains('Breast')

0       False
1       False
2       False
3       False
4       False
        ...  
1824    False
1825    False
1826    False
1827    False
1828    False
Name: disease_subtype, Length: 1829, dtype: object

In [164]:
dep['primary_disease'].unique()

array(['Ovarian Cancer', 'Leukemia', 'Colon/Colorectal Cancer',
       'Skin Cancer', 'Bladder Cancer', 'Lung Cancer', 'Kidney Cancer',
       'Breast Cancer', 'Pancreatic Cancer', 'Myeloma', 'Brain Cancer',
       'Sarcoma', 'Lymphoma', 'Bone Cancer', 'Fibroblast',
       'Gastric Cancer', 'Engineered', 'Thyroid Cancer', 'Neuroblastoma',
       'Prostate Cancer', 'Rhabdoid', 'Gallbladder Cancer',
       'Endometrial/Uterine Cancer', 'Head and Neck Cancer',
       'Bile Duct Cancer', 'Esophageal Cancer', 'Liver Cancer',
       'Cervical Cancer', 'Unknown', 'Eye Cancer', 'Adrenal Cancer',
       'Liposarcoma', 'Embryonal Cancer', 'Teratoma', 'Non-Cancerous'],
      dtype=object)

In [165]:
dep['disease_subtype'].unique()

array(['Adenocarcinoma, high grade serous',
       'Acute Myelogenous Leukemia (AML), M3 (Promyelocytic)',
       'Adenocarcinoma',
       'Acute Myelogenous Leukemia (AML), M6 (Erythroleukemia)',
       'Acute Myelogenous Leukemia (AML), M5 (Eosinophilic/Monocytic)',
       'Melanoma', 'Carcinoma',
       'Non-Small Cell Lung Cancer (NSCLC), Adenocarcinoma',
       'Non-Small Cell Lung Cancer (NSCLC), Large Cell Carcinoma',
       'Renal Cell Carcinoma',
       'Acute Lymphoblastic Leukemia (ALL), B-cell',
       'Ductal Adenocarcinoma, exocrine', 'Multiple Myeloma',
       'Meningioma', 'Glioblastoma', 'Acute Myelogenous Leukemia (AML)',
       'Thyroid Sarcoma', 'B-cell', 'Ewings Sarcoma', 'Astrocytoma',
       'Skin', 'Breast Ductal Carcinoma', 'Adenocarcinoma, endometrioid',
       nan, 'Multiple Myeloma, plasmacytoma', 'Rhabdomyosarcoma',
       'T-cell, Non-Hodgkins, Anaplastic Large Cell (ALCL)',
       'Fibrosarcoma', 'Medulloblastoma',
       'Diffuse Large B-cell Lymphoma (D

In [167]:
meta_tcga['cancer_type'].unique()

array(['uveal melanoma', 'uterine corpus endometrioid carcinoma',
       'uterine carcinosarcoma', 'thyroid carcinoma', 'thymoma',
       'testicular germ cell tumor', 'stomach adenocarcinoma',
       'skin cutaneous melanoma', 'sarcoma', 'rectum adenocarcinoma',
       'prostate adenocarcinoma', 'pheochromocytoma & paraganglioma',
       'pancreatic adenocarcinoma', 'ovarian serous cystadenocarcinoma',
       'mesothelioma', 'lung squamous cell carcinoma',
       'lung adenocarcinoma', 'liver hepatocellular carcinoma',
       'kidney papillary cell carcinoma', 'kidney clear cell carcinoma',
       'kidney chromophobe', 'head & neck squamous cell carcinoma',
       'glioblastoma multiforme', 'esophageal carcinoma',
       'diffuse large b-cell lymphoma', 'colon adenocarcinoma',
       'cholangiocarcinoma', 'cervical & endocervical cancer',
       'breast invasive carcinoma', 'brain lower grade glioma',
       'bladder urothelial carcinoma', 'adrenocortical cancer'],
      dtype=object)

In [168]:
gdc

,full_name,abbreviation
0,acute myeloid leukemia,LAML
1,adrenocortical cancer,ACC
2,bladder urothelial carcinoma,BLCA
3,brain lower grade glioma,LGG
4,breast invasive carcinoma,BRCA
5,cervical & endocervical cancer,CESC
6,cholangiocarcinoma,CHOL
7,chronic myelogenous leukemia,LCML
8,colon adenocarcinoma,COAD
9,esophageal carcinoma,ESCA


In [170]:
dep.to_csv('../data/processed/depmap_22q1_cellline_info_140722.csv', index=False)

In [2]:
gtex = pd.read_csv('../data/processed/toil_gtex_sample_info_140722.csv')

In [3]:
gtex

,primary_site,detailed_site,sample,gender
0,cervix,cervix - endocervix,GTEX-S341-1326-SM-4AD72,female
1,cervix,cervix - endocervix,GTEX-T6MO-1426-SM-4DM73,female
2,cervix,cervix - endocervix,GTEX-S32W-1626-SM-4AD6G,female
3,cervix,cervix - endocervix,GTEX-TSE9-2726-SM-4DXSQ,female
4,cervix,cervix - ectocervix,GTEX-S4UY-1426-SM-4AD6Y,female
...,...,...,...,...
7420,adipose tissue,adipose - subcutaneous,GTEX-POMQ-2326-SM-2S1O8,female
7421,adipose tissue,adipose - subcutaneous,GTEX-QVJO-0226-SM-2S1R2,female
7422,adipose tissue,adipose - subcutaneous,GTEX-P4PP-0226-SM-2S1NN,female
7423,adipose tissue,adipose - subcutaneous,GTEX-NL3G-0226-SM-4RGLU,female


In [4]:
gtex['detailed_site'] = gtex['detailed_site'].map(lambda x: x[x.index('-')+2:] if '-' in x else x)

In [5]:
gtex['detailed_site'].unique()

array(['endocervix', 'ectocervix', 'bladder', 'fallopian tube',
       'amygdala', 'spinal cord (cervical c-1)', 'hypothalamus',
       'putamen (basal ganglia)', 'nucleus accumbens (basal ganglia)',
       'caudate (basal ganglia)', 'cerebellar hemisphere',
       'frontal cortex (ba9)', 'anterior cingulate cortex (ba24)',
       'substantia nigra', 'hippocampus', 'cortex', 'liver', 'aorta',
       'whole blood', 'cerebellum', 'pituitary', 'left ventricle',
       'tibial', 'sun exposed (lower leg)', 'testis', 'prostate',
       'transverse', 'terminal ileum', 'sigmoid', 'stomach',
       'gastroesophageal junction', 'mucosa', 'muscularis', 'pancreas',
       'spleen', 'lung', 'thyroid', 'adrenal gland',
       'minor salivary gland', 'not sun exposed (suprapubic)',
       'mammary tissue', 'vagina', 'uterus', 'ovary',
       'visceral (omentum)', 'atrial appendage', 'coronary', 'skeletal',
       'subcutaneous'], dtype=object)

In [6]:
gtex_groups = gtex.groupby(['primary_site', 'detailed_site']).size().to_frame()

In [7]:
gtex_groups = gtex_groups.rename({0: 'samples'}, axis=1).reset_index()

In [8]:
gtex['primary_site'] = gtex['primary_site'].map()

,primary_site,detailed_site,samples
0,adipose tissue,subcutaneous,318
1,adipose tissue,visceral (omentum),197
2,adrenal gland,adrenal gland,128
3,bladder,bladder,9
4,blood,whole blood,337
5,blood vessel,aorta,207
6,blood vessel,coronary,118
7,blood vessel,tibial,281
8,brain,amygdala,69
9,brain,anterior cingulate cortex (ba24),83


In [12]:
for i, j in enumerate(gtex['primary_site'].unique()):
    samples = gtex.loc[gtex['primary_site'] == j, 'sample'].to_list()
    toil_part = pd.read_csv('../data/raw/toil_expression_trs_260422', sep='\t', usecols=['sample']+samples).set_index('sample')
    toil_part = np.around(2 ** toil_part - 0.001, decimals=5)
    toil_part = toil_part.where(toil_part != -0.0, 0.0)
    toil_part.to_csv(f'../data/processed/toil_parts/toil_gtex_{j.replace(" ", "_")}_tpm.csv')
    print(f'{i+1} / {gtex["primary_site"].nunique()}   ', end='\r')

In [10]:
gtex_groups.to_csv('../data/processed/toil_gtex_group_size_150722.csv', index=False)

In [18]:
toil = pd.read_csv('../data/raw/toil_expression_trs_260422', sep='\t', nrows=2)

In [23]:
tcga = pd.read_csv('../data/processed/toil_tcga_sample_info_140722.csv')

In [9]:
gtex

,primary_site,detailed_site,sample,gender
0,cervix,endocervix,GTEX-S341-1326-SM-4AD72,female
1,cervix,endocervix,GTEX-T6MO-1426-SM-4DM73,female
2,cervix,endocervix,GTEX-S32W-1626-SM-4AD6G,female
3,cervix,endocervix,GTEX-TSE9-2726-SM-4DXSQ,female
4,cervix,ectocervix,GTEX-S4UY-1426-SM-4AD6Y,female
...,...,...,...,...
7420,adipose tissue,subcutaneous,GTEX-POMQ-2326-SM-2S1O8,female
7421,adipose tissue,subcutaneous,GTEX-QVJO-0226-SM-2S1R2,female
7422,adipose tissue,subcutaneous,GTEX-P4PP-0226-SM-2S1NN,female
7423,adipose tissue,subcutaneous,GTEX-NL3G-0226-SM-4RGLU,female


In [25]:
toil

,sample,GTEX-S4Q7-0003-SM-3NM8M,TCGA-19-1787-01,TCGA-S9-A7J2-01,GTEX-QV31-1626-SM-2S1QC,TCGA-G3-A3CH-11,GTEX-13OVI-1026-SM-5L3EM,GTEX-13OW5-0626-SM-5J2N2,GTEX-XUZC-2026-SM-4BRW9,TCGA-EK-A2RE-01,TCGA-D5-5538-01,TCGA-VM-A8C8-01,GTEX-X88G-0426-SM-47JZ5,TCGA-ZN-A9VQ-01,TCGA-EE-A17X-06,GTEX-ZAKK-0326-SM-5EGHA,GTEX-1399S-1726-SM-5L3DI,GTEX-ZF29-0926-SM-4WKFZ,TCGA-05-4420-01,GTEX-13W3W-2626-SM-5SIA2,GTEX-WQUQ-2526-SM-4MVNO,TCGA-2G-AAFZ-01,GTEX-P44H-0526-SM-2XCF1,TCGA-DX-A3U8-01,TCGA-GL-8500-01,...,GTEX-R55E-0008-SM-48FCG,TCGA-YL-A9WK-01,TCGA-77-6844-01,TCGA-B8-5549-01,TCGA-IN-AB1X-11,TCGA-CS-4944-01,TCGA-A2-A0YK-01,TCGA-AA-3660-01,TCGA-13-1481-01,GTEX-1399R-0426-SM-5IJE3,GTEX-YFCO-1626-SM-4W1Z3,TCGA-DJ-A2QC-01,TCGA-A8-A09K-01,TCGA-61-1907-01,GTEX-X3Y1-1226-SM-4PQZ8,TCGA-IB-7885-01,TCGA-B6-A0IA-01,GTEX-1117F-2826-SM-5GZXL,TCGA-VQ-AA6F-01,TCGA-BR-8588-01,GTEX-11ZTS-3326-SM-5LU9Y,TCGA-DD-A115-01,GTEX-ZA64-2126-SM-5Q5A8,TCGA-FV-A3I0-11,GTEX-XV7Q-0426-SM-4BRVN
0,ENST00000548312.5,0.0300,-0.5125,-4.0350,-9.9658,-5.0116,-9.9658,-2.5479,-9.9658,-5.0116,-3.8160,-2.1140,-3.3076,-1.9942,-9.9658,-2.5479,-2.8262,-9.9658,-3.4580,-9.9658,-3.1714,-1.1488,-2.7274,-0.6193,-3.1714,...,-3.0469,-2.1779,-9.9658,-1.2828,-9.9658,-2.8262,-1.8836,-3.4580,-5.5735,-9.9658,-4.2934,-2.5479,-3.4580,-3.4580,-1.9942,-2.1140,-5.5735,-9.9658,-3.6259,-9.9658,-1.9379,-3.3076,-9.9658,-3.8160,-9.9658
1,ENST00000483781.5,-9.9658,-9.9658,-5.0116,-3.3076,-9.9658,-1.6394,-9.9658,-9.9658,-5.0116,-9.9658,-9.9658,-0.7108,-9.9658,-9.9658,-9.9658,0.2762,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-1.9379,-3.6259,-9.9658,...,-9.9658,-3.6259,-2.7274,-9.9658,-3.6259,-9.9658,-1.7809,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.9324,-2.4659,-9.9658,-9.9658,-1.7322,-1.7809,-0.4131,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [32]:
%%time
toil = pd.read_csv('../data/raw/toil_expression_trs_260422', sep='\t', usecols=['sample', 'TCGA-19-1787-01', 'TCGA-S9-A7J2-01'])

CPU times: total: 2min 56s
Wall time: 3min 25s


In [29]:
toil

,sample,TCGA-19-1787-01,TCGA-S9-A7J2-01
0,ENST00000548312.5,-0.5125,-4.0350
1,ENST00000483781.5,-9.9658,-5.0116
2,ENST00000535093.1,-9.9658,-9.9658
3,ENST00000338863.11,0.0990,2.0742
4,ENST00000570899.1,-2.4659,-9.9658
...,...,...,...
198614,ENST00000402231.1,-9.9658,-9.9658
198615,ENST00000463473.2,-9.9658,-9.9658
198616,ENST00000380293.3,-9.9658,-9.9658
198617,ENST00000288710.6,0.7233,-6.5064


In [30]:
import polars as pl

In [33]:
%%time
pl.read_csv('../data/raw/toil_expression_trs_260422', sep='\t', columns=['sample', 'TCGA-19-1787-01', 'TCGA-S9-A7J2-01'])

CPU times: total: 1min 26s
Wall time: 1min 50s


sample,TCGA-19-1787-01,TCGA-S9-A7J2-01
str,f64,f64
"""ENST00000548312.5""",-0.5125,-4.035
"""ENST00000483781.5""",-9.9658,-5.0116
"""ENST00000535093.1""",-9.9658,-9.9658
"""ENST00000338863.11""",0.099,2.0742
"""ENST00000570899.1""",-2.4659,-9.9658
"""ENST00000556831.5""",-0.8339,-0.1031
"""ENST00000625998.2""",1.0293,2.0147
"""ENST00000583693.5""",0.2277,1.8683
"""ENST00000580812.1""",-0.2159,-1.3548


In [59]:
for i, j in enumerate(tcga['project'].unique()):
    samples = tcga.loc[tcga['project'] == j, 'sample'].to_list()
    toil_part = pd.read_csv('../data/raw/toil_expression_trs_260422', sep='\t', usecols=['sample']+samples).set_index('sample')
    toil_part = np.around(2 ** toil_part - 0.001, decimals=5)
    toil_part = toil_part.where(toil_part != -0.0, 0.0)
    toil_part.to_csv(f'../data/processed/toil_parts/toil_tcga_{j.lower()}_tpm.csv')
    print(f'{i+1} / {tcga["project"].nunique()}   ', end='\r')

In [56]:
samples = tcga.loc[tcga['project'] == 'UVM', 'sample'].to_list()
toil_part = pd.read_csv('../data/raw/toil_expression_trs_260422', sep='\t', usecols=['sample']+samples).set_index('sample')
toil_part = np.around(2 ** toil_part - 0.001, decimals=5)
toil_part = toil_part.where(toil_part != -0.0, 0.0)

In [46]:
toil_part.to_csv('../data/processed/toil_parts/uvm_test.csv')

In [49]:
toil_part = pd.read_csv('../data/processed/toil_parts/uvm_test.csv', index_col=0)

In [54]:
toil_part[(toil_part != 0).any(1)]

,TCGA-VD-A8KF-01,TCGA-VD-A8KE-01,TCGA-WC-A88A-01,TCGA-VD-A8KA-01,TCGA-WC-A884-01,TCGA-VD-A8KK-01,TCGA-V4-A9F7-01,TCGA-V4-A9EQ-01,TCGA-V4-A9EF-01,TCGA-V4-A9EI-01,TCGA-YZ-A984-01,TCGA-V4-A9EZ-01,TCGA-VD-A8KH-01,TCGA-V4-A9E5-01,TCGA-VD-AA8R-01,TCGA-WC-A885-01,TCGA-WC-A87W-01,TCGA-V4-A9ES-01,TCGA-V4-A9E8-01,TCGA-V4-A9EW-01,TCGA-V4-A9F8-01,TCGA-V4-A9EV-01,TCGA-VD-AA8P-01,TCGA-V4-A9EK-01,TCGA-VD-A8KD-01,...,TCGA-V4-A9ED-01,TCGA-V4-A9F4-01,TCGA-YZ-A982-01,TCGA-WC-A87T-01,TCGA-VD-A8K9-01,TCGA-YZ-A985-01,TCGA-VD-A8KJ-01,TCGA-VD-A8KB-01,TCGA-V4-A9EL-01,TCGA-VD-A8KN-01,TCGA-RZ-AB0B-01,TCGA-WC-A883-01,TCGA-VD-AA8S-01,TCGA-WC-A87Y-01,TCGA-V4-A9F1-01,TCGA-V4-A9EY-01,TCGA-V4-A9EU-01,TCGA-WC-A87U-01,TCGA-VD-AA8O-01,TCGA-VD-A8KO-01,TCGA-WC-A881-01,TCGA-V4-A9ET-01,TCGA-WC-AA9E-01,TCGA-V4-A9EC-01,TCGA-V4-A9EH-01
sample,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENST00000548312.5,0.3100,0.1000,0.1700,0.0400,0.2400,0.0300,0.2100,0.43,0.1100,0.0300,0.1000,0.1600,0.0300,-0.0000,0.2400,0.1400,0.5100,0.0700,0.31,0.12,0.2100,0.19,0.48,0.10,0.1300,...,0.3000,0.2900,0.3200,0.14,0.1100,0.0700,-0.0000,0.2000,0.1900,0.2400,0.1400,0.1100,0.1900,0.0800,0.2400,0.2600,0.1800,0.1100,0.1700,-0.0000,0.0600,0.2600,0.21,0.0700,0.3600
ENST00000483781.5,0.0300,-0.0000,-0.0000,0.0400,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0200,0.04,-0.00,-0.0000,0.33,-0.00,0.07,-0.0000,...,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0500,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0300,-0.00,0.0300,-0.0000
ENST00000535093.1,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.00,-0.0000,-0.00,-0.00,-0.00,-0.0000,...,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000
ENST00000338863.11,6.8599,9.1498,3.8999,15.2698,9.3401,2.3299,10.5397,7.36,6.1401,9.0101,5.1698,5.3001,4.3099,14.5899,6.5399,8.0001,14.0403,6.7402,6.42,6.96,6.1201,4.59,7.68,14.09,4.3001,...,7.9102,11.3198,20.1894,4.23,10.9304,5.3101,6.2501,9.5503,4.4299,7.8599,4.8299,4.9499,21.3999,5.2199,4.9301,6.7899,7.6699,6.8201,7.8599,5.4299,2.7299,14.7998,6.27,3.5801,9.3999
ENST00000570899.1,0.1200,0.2900,0.1000,0.1000,0.2300,-0.0000,-0.0000,0.26,0.2000,0.1800,-0.0000,-0.0000,0.0800,-0.0000,-0.0000,0.1800,0.2300,-0.0000,0.23,-0.00,-0.0000,0.22,0.13,0.29,0.1100,...,0.1200,-0.0000,0.4300,-0.00,0.0900,-0.0000,0.0900,-0.0000,-0.0000,-0.0000,0.1300,-0.0000,0.1400,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.1500,-0.0000,-0.0000,-0.0000,0.08,0.1000,-0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENST00000465576.1,-0.0000,0.0100,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.00,-0.0000,-0.00,0.02,0.01,-0.0000,...,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0100,-0.0000,-0.0000,0.0800,0.0100,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000
ENST00000463473.2,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.00,-0.0000,-0.00,-0.00,0.06,0.0400,...,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000,-0.0000,-0.0000,0.0500,0.0700,0.0400,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0300,-0.00,-0.0000,-0.0000
ENST00000380293.3,-0.0000,-0.0000,0.0400,-0.0000,-0.0000,-0.0000,-0.0000,-0.00,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0500,-0.0000,-0.00,-0.00,-0.0000,-0.00,-0.00,-0.00,-0.0000,...,-0.0000,-0.

In [57]:
toil_part

,TCGA-VD-A8KF-01,TCGA-VD-A8KE-01,TCGA-WC-A88A-01,TCGA-VD-A8KA-01,TCGA-WC-A884-01,TCGA-VD-A8KK-01,TCGA-V4-A9F7-01,TCGA-V4-A9EQ-01,TCGA-V4-A9EF-01,TCGA-V4-A9EI-01,TCGA-YZ-A984-01,TCGA-V4-A9EZ-01,TCGA-VD-A8KH-01,TCGA-V4-A9E5-01,TCGA-VD-AA8R-01,TCGA-WC-A885-01,TCGA-WC-A87W-01,TCGA-V4-A9ES-01,TCGA-V4-A9E8-01,TCGA-V4-A9EW-01,TCGA-V4-A9F8-01,TCGA-V4-A9EV-01,TCGA-VD-AA8P-01,TCGA-V4-A9EK-01,TCGA-VD-A8KD-01,...,TCGA-V4-A9ED-01,TCGA-V4-A9F4-01,TCGA-YZ-A982-01,TCGA-WC-A87T-01,TCGA-VD-A8K9-01,TCGA-YZ-A985-01,TCGA-VD-A8KJ-01,TCGA-VD-A8KB-01,TCGA-V4-A9EL-01,TCGA-VD-A8KN-01,TCGA-RZ-AB0B-01,TCGA-WC-A883-01,TCGA-VD-AA8S-01,TCGA-WC-A87Y-01,TCGA-V4-A9F1-01,TCGA-V4-A9EY-01,TCGA-V4-A9EU-01,TCGA-WC-A87U-01,TCGA-VD-AA8O-01,TCGA-VD-A8KO-01,TCGA-WC-A881-01,TCGA-V4-A9ET-01,TCGA-WC-AA9E-01,TCGA-V4-A9EC-01,TCGA-V4-A9EH-01
sample,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENST00000548312.5,0.31000,0.10000,0.17000,0.04000,0.24000,0.03000,0.21000,0.43001,0.11000,0.03000,0.10000,0.16000,0.03000,0.00000,0.24000,0.14000,0.51000,0.07000,0.31000,0.12000,0.21000,0.18999,0.48000,0.10000,0.13000,...,0.29999,0.29000,0.31999,0.14000,0.11000,0.07000,0.00000,0.20000,0.18999,0.24000,0.14000,0.11000,0.18999,0.08000,0.24000,0.26000,0.18000,0.11000,0.17000,0.00000,0.06000,0.26000,0.21000,0.07000,0.36001
ENST00000483781.5,0.03000,0.00000,0.00000,0.04000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.02000,0.04000,0.00000,0.00000,0.33000,0.00000,0.07000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.05000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.03000,0.00000,0.03000,0.00000
ENST00000535093.1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
ENST00000338863.11,6.85991,9.14981,3.89988,15.26976,9.34015,2.32992,10.53971,7.35999,6.14011,9.01008,5.16984,5.30013,4.30993,14.58993,6.53995,8.00011,14.04026,6.74017,6.42001,6.96003,6.12014,4.58997,7.68003,14.08998,4.30008,...,7.91022,11.31977,20.18943,4.23000,10.93035,5.31006,6.25006,9.55032,4.42990,7.85993,4.82993,4.94993,21.39991,5.21991,4.93007,6.78989,7.66992,6.82008,7.85993,5.42993,2.72994,14.79977,6.27002,3.58009,9.39991
ENST00000570899.1,0.12000,0.29000,0.10000,0.10000,0.23001,0.00000,0.00000,0.26000,0.20000,0.18000,0.00000,0.00000,0.08000,0.00000,0.00000,0.18000,0.23001,0.00000,0.23001,0.00000,0.00000,0.22000,0.13000,0.29000,0.11000,...,0.12000,0.00000,0.43001,0.00000,0.09000,0.00000,0.09000,0.00000,0.00000,0.00000,0.13000,0.00000,0.14000,0.00000,0.00000,0.00000,0.00000,0.00000,0.15000,0.00000,0.00000,0.00000,0.08000,0.10000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENST00000402231.1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
ENST00000463473.2,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.06000,0.04000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.05000,0.07000,0.04000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00

In [15]:
dep = pd.read_csv('../data/custom/depmap_22q1_expression_trs_processed_240422.csv')

In [16]:
dep

,gene_name,enst_id,143B,22RV1,2313287,253J,42MGBA,5637,59M,639V,...,YAMATO,YAPC,YD15,YD38,YD8,YH13,YKG1,YSCCC,ZR751,trs_type
0,A1BG,ENST00000263100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,protein_coding
1,A1BG,ENST00000595014,0.014355,0.464668,0.000000,0.495695,0.298658,0.014355,0.250962,0.400538,...,0.056584,0.000000,0.000000,0.000000,0.298658,0.000000,0.687061,0.137504,0.956057,NaN
2,A1BG,ENST00000596924,0.070389,0.201634,0.000000,0.014355,0.575312,0.000000,0.505891,0.464668,...,0.189034,0.000000,0.000000,0.000000,0.000000,0.632268,0.555816,0.070389,0.475085,NaN
3,A1BG,ENST00000598345,3.016140,1.469886,0.189034,0.565597,5.630522,0.226509,4.932628,4.958843,...,4.004501,0.000000,0.389567,0.411426,0.933573,4.761285,4.351204,4.751678,5.528259,NaN
4,A1BG,ENST00000600966,0.000000,0.000000,0.000000,0.189034,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.367371,0.000000,0.000000,0.000000,0.084064,0.000000,0.000000,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140894,ZZZ3,ENST00000469944,0.000000,0.070389,0.536053,0.000000,0.201634,0.000000,0.111031,0.111031,...,0.000000,0.097611,0.000000,0.137504,0.748461,0.000000,0.367371,0.000000,0.604071,NaN
140895,ZZZ3,ENST00000474746,0.124328,0.000000,0.000000,0.000000,0.214125,0.000000,0.000000,0.000000,...,0.000000,0.298658,0.000000,0.000000,0.000000,1.731183,0.286881,0.000000,0.000000,NaN
140896,ZZZ3,ENST00000476195,0.000000,0.163499,0.111031,0.111031,0.250962,0.226509,0.201634,0.070389,...,0.731183,0.000000,0.000000,0.000000,0.275007,0.000000,0.189034,0.176323,0.000000,NaN
140897,ZZZ3,ENST00000476275,0.097611,0.298658,0.748461,0.137504,0.704872,0.084064,0.286881,0.389567,...,0.028569,0.250962,0.464668,0.584963,0.613532,1.541019,0.454176,0.163499,0.604071,NaN


In [3]:
g = glob.glob('../data/processed/toil/*.csv')

In [9]:
for i, j in enumerate(g[1:], start=1):
    x = pd.read_csv(j, index_col=0)
    x.index = x.index.map(lambda x: x[:x.index('.')])
    x.to_csv(j)
    print(f'{i} / {len(g)}   ', end='\r')

In [11]:
pd.read_csv(g[40], index_col=0, nrows=3)

,TCGA-KL-8324-11,TCGA-KL-8345-01,TCGA-KL-8346-01,TCGA-KN-8419-11,TCGA-KL-8342-01,TCGA-KN-8435-01,TCGA-KN-8436-01,TCGA-KN-8427-01,TCGA-KN-8428-11,TCGA-KN-8424-11,...,TCGA-KO-8408-01,TCGA-KL-8343-01,TCGA-KL-8327-01,TCGA-KO-8407-01,TCGA-KN-8433-01,TCGA-KO-8403-11,TCGA-KO-8416-01,TCGA-KL-8323-01,TCGA-KN-8429-11,TCGA-KL-8332-01
sample,,,,,,,,,,,,,,,,,,,,,
ENST00000548312,0.09,0.14,0.1,0.09,0.03,0.17,0.09,0.04000,0.15,0.04,...,0.03,0.09,0.07,0.08,0.0,0.12,0.06,0.07,0.18999,0.15
ENST00000483781,0.09,0.10,0.0,0.00,0.00,0.04,0.12,0.18999,0.00,0.00,...,0.02,0.00,0.00,0.00,0.0,0.03,0.00,0.00,0.00000,0.00
ENST00000535093,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00000,0.00
